---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np

def blight_model():
    return answer


Memory Problems:

The best way to avoid memory problems is to create two versions of the assignment notebook, one for development 
and one for submission. To avoid memory issues: 

1. Read only the columns you use in the final model, see read_csv has very useful parameter that can help reduce 
processing 
    usecols: provide a list of all columns to be read
    
    index_col: provide the name of the column to be used as index
    
    parse_dates: provide a list of columns that needs to be parsed)
    
    dtypes: provide a dictionary of column names as keys and types as values, use ‘str’ for string,’float’ for
    float64, don’t use integer as NaN values are not allowed in integer data)
    
    converters: these can be useful for applying simple transformations to the columns, one example 
    {‘col_name’:lambda x: x.upper() if type(x)==str else x} can be used to convert all string items to upper case. 
    Only use with mostly non-numeric data
    
2. Replace or delete NaN values as early as possible in your code, use the same column datatype to fill each column, 
    e.g. if the column dtype is float64, use a float number, if the column dtype is DateTime or TimeStamp use 
    pd.to_datetime(‘Date here’) and if the dtype is Timedelta use pd.Timedelta(‘number units’) units can days, 
    months, etc..

3. Remove redundant features, e.g. a column containing only one distinct value will add little information to your 
    learning model

4. Keep your code neat, create functions to perform repetitive tasks, e.g. once you decided on the columns to use 
    and how to deal with NaN values and categorical data, create a function that you can use for both train and test

5. Run your code on the online platform before submission, if it takes too long to run the code or restarts kernel, 
    this is an indication that the code needs modification/optimising. One example, you want to divide a pandas 
    series (ser) by a constant C, instead of division ser/C use multiplication ser*(1/C)

How to calculate AUC:

train.csv is the only data available a the point when the model is designed, so to determine your learning model, 
and its parameters, you need to use train/test split or more preferably to avoid overfilling; cross_validation

the test.csv is to be used only to predict the probability of class 1(i.e. blight ticket paid on time)

If you achieve a high AUC on the training data but the grader returns a low AUC, you'll need to look at the model 
and whether it's being overfitted. the main cause of overfitting is the training data being very different from 
test data, that's why cross validation is important (I used Gridsearch setting scoring to 'roc_auc to choose my model')

A second reason for overfitting is low number of training point, dropping all rows that contain a NaN value may result 
in much fewer data points that you would expect, that's why it's important to display your data at each step in the 
design stage.


Things to try when grader's AUC is too low

--different learning model, different parameters, use cross validation or multiple values of random_state when 
splitting train/test

--add/remove features; you can achieve passing grader with even when using 2 features.

--do you use scaling? is the method you're using suitable for this application? try a different scaler

In [2]:
import pandas as pd
import numpy as np

df_train = pd.read_csv('/Users/karenyang/Desktop/DataScience_MachineLearning_Python/week_4_SupervisedML_part2/train.csv', 
                       encoding="latin1", low_memory=False)
df_train.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [3]:
df_train.shape  # 250,306 examples with 34 columns

(250306, 34)

In [4]:
# See how many nulls there are for each feature in the dataframe
df_train.isnull().sum()

ticket_id                          0
agency_name                        0
inspector_name                     0
violator_name                     34
violation_street_number            0
violation_street_name              0
violation_zip_code            250306
mailing_address_str_number      3602
mailing_address_str_name           4
city                               0
state                             93
zip_code                           1
non_us_str_code               250303
country                            0
ticket_issued_date                 0
hearing_date                   12491
violation_code                     0
violation_description              0
disposition                        0
fine_amount                        1
admin_fee                          0
state_fee                          0
late_fee                           0
discount_amount                    0
clean_up_cost                      0
judgment_amount                    0
payment_amount                     0
b

In [5]:
# Drop some features that are not of interest
df_train.drop(['violation_zip_code','grafitti_status','inspector_name','violator_name', 'violation_street_number',
         'violation_street_name','violation_zip_code','non_us_str_code', 'mailing_address_str_number',
         'mailing_address_str_name', 'payment_date','payment_status','collection_status'],inplace=True,axis=1)



In [6]:
# See how many nulls there are for each feature in the dataframe
df_train.isnull().sum()

ticket_id                    0
agency_name                  0
city                         0
state                       93
zip_code                     1
country                      0
ticket_issued_date           0
hearing_date             12491
violation_code               0
violation_description        0
disposition                  0
fine_amount                  1
admin_fee                    0
state_fee                    0
late_fee                     0
discount_amount              0
clean_up_cost                0
judgment_amount              0
payment_amount               0
balance_due                  0
compliance_detail            0
compliance               90426
dtype: int64

In [7]:
# filter the dataframe to only include rows in which hearing_date is not null
df_train = df_train[df_train.hearing_date.notnull()]

In [8]:
# if judgment_amount is 0.0 then set compliance equal to 1 for 90426 values to retain as many examples as possible
df_train['compliance'][df_train.judgment_amount == 0.0] = 1.0

In [9]:
df_train.isnull().sum()

ticket_id                 0
agency_name               0
city                      0
state                    93
zip_code                  0
country                   0
ticket_issued_date        0
hearing_date              0
violation_code            0
violation_description     0
disposition               0
fine_amount               1
admin_fee                 0
state_fee                 0
late_fee                  0
discount_amount           0
clean_up_cost             0
judgment_amount           0
payment_amount            0
balance_due               0
compliance_detail         0
compliance                0
dtype: int64

In [10]:
# One example has NaN so reset value to 0
df_train[['fine_amount']] = df_train[['fine_amount']].fillna(value=0)

In [11]:
df_train.isnull().sum()

ticket_id                 0
agency_name               0
city                      0
state                    93
zip_code                  0
country                   0
ticket_issued_date        0
hearing_date              0
violation_code            0
violation_description     0
disposition               0
fine_amount               0
admin_fee                 0
state_fee                 0
late_fee                  0
discount_amount           0
clean_up_cost             0
judgment_amount           0
payment_amount            0
balance_due               0
compliance_detail         0
compliance                0
dtype: int64

In [12]:
# inspect the target variable, compliance
#compliance [target variable for prediction] 
# Null = Not responsible
# 0 = Responsible, non-compliant
# 1 = Responsible, compliant
# For this assignment, your task is to predict whether a given blight ticket will be paid on time.
df_train.compliance.value_counts()

0.0    148222
1.0     89593
Name: compliance, dtype: int64

In [13]:
df_train.compliance.count()

237815

In [14]:
# agency names for blight ticket in the dataset
#df_train.agency_name.unique()
df_train.agency_name.value_counts()

Buildings, Safety Engineering & Env Department    149037
Department of Public Works                         71464
Detroit Police Department                           8763
Health Department                                   8549
Neighborhood City Halls                                2
Name: agency_name, dtype: int64

In [15]:
# code violations in the particular dataset; too many to make categorical
df_train.violation_code.value_counts()

9-1-36(a)               93808
9-1-81(a)               41052
22-2-88                 27231
9-1-104                 21725
22-2-88(b)               6882
22-2-45                  5223
9-1-43(a) - (Dwellin     4959
9-1-105                  4923
9-1-110(a)               4684
22-2-22                  3592
9-1-103(C)               3442
22-2-43                  2805
19450901                 2769
22-2-17                  1875
22-2-61                  1395
22-2-83(a)(b)(c)          807
61-81.0100/32.0066        799
9-1-82(d) - (Dwellin      791
9-1-43(a) - (Structu      727
22-2-83                   604
61-5-21                   407
9-1-206                   335
22-2-88(a)                332
9-1-43(a) - (Stories      324
9-1-209                   321
9-1-101                   287
22-2-21(b)                276
61-5-18                   262
9-1-107                   218
9-1-111                   200
                        ...  
9-1-381                     1
9-1-354                     1
9-1-444   

In [16]:
# df_train.judgment_amount.max()  
df_train.judgment_amount.median()  

250.0

In [17]:
df_train.disposition.value_counts()

Responsible by Default                138234
Not responsible by Dismissal           47406
Not responsible by City Dismissal      23820
Responsible by Admission               13666
Responsible by Determination            7563
Not responsible by Determination        6542
PENDING JUDGMENT                         387
Responsible (Fine Waived) by Deter       190
SET-ASIDE (PENDING JUDGMENT)               7
Name: disposition, dtype: int64

In [18]:
df_train.columns

Index(['ticket_id', 'agency_name', 'city', 'state', 'zip_code', 'country',
       'ticket_issued_date', 'hearing_date', 'violation_code',
       'violation_description', 'disposition', 'fine_amount', 'admin_fee',
       'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost',
       'judgment_amount', 'payment_amount', 'balance_due', 'compliance_detail',
       'compliance'],
      dtype='object')

In [19]:
# convert to int to find the number of days from ticket_issued_date to the hearing_date
df_train['time_to_hearing_date'] = (pd.to_datetime(df_train['hearing_date']).dt.date - 
                     pd.to_datetime(df_train['ticket_issued_date']).dt.date).fillna(pd.Timedelta('-1 days')).dt.days

In [20]:
df_train['time_to_hearing_date'].head()

0    370
1    378
2    337
4    337
5    324
Name: time_to_hearing_date, dtype: int64

In [21]:
df_train.fine_amount.median()

250.0

In [22]:
df_train.admin_fee.max()

20.0

In [23]:
df_train.state_fee.max()

10.0

In [24]:
df_train.late_fee.max()

1000.0

In [25]:
df_train.discount_amount.max()

350.0

In [26]:
df_train.clean_up_cost.median()

0.0

In [27]:
from sklearn import preprocessing

# Create categorical variable for agency_name
le = preprocessing.LabelEncoder()
an = df_train['agency_name'].tolist()
le_model = le.fit_transform(an)
# print(type(le_model))
df_train['agency_name'] = le_model
#df_train['agency_name'].unique()

In [28]:
# Create categorical variable for disposition
le2 = preprocessing.LabelEncoder()
dis = df_train['disposition'].tolist()
le_model2 = le2.fit_transform(dis)

df_train['disposition'] = le_model2
#df_train['disposition'].unique()

In [29]:
# Set up X_train, y_train for training dataset  # 237815 examples  
X_train = df_train[['time_to_hearing_date','fine_amount', 'admin_fee','state_fee', 'late_fee','discount_amount', 'judgment_amount']]
y_train = df_train['compliance']
y_train.count()

237815

In [30]:
# Read in test dataset; no y_values in dataset, grader has y_test data
# /Users/karenyang/Desktop/DataScience_MachineLearning_Python/week_4_SupervisedML_part2/

df_test = pd.read_csv('/Users/karenyang/Desktop/DataScience_MachineLearning_Python/week_4_SupervisedML_part2/test.csv', 
          encoding="latin1",  
          low_memory=False,
          index_col=['ticket_id'],    
          usecols=['agency_name', 'ticket_id', 'ticket_issued_date', 'hearing_date', 'fine_amount', 'admin_fee', 
                    'state_fee', 'late_fee', 'discount_amount','disposition', 'judgment_amount'])



In [31]:
df_test.shape  

(61001, 10)

In [32]:
# See how many nulls there are for each feature in the dataframe
df_test.isnull().sum()

agency_name              0
ticket_issued_date       0
hearing_date          2197
disposition              0
fine_amount              0
admin_fee                0
state_fee                0
late_fee                 0
discount_amount          0
judgment_amount          0
dtype: int64

In [33]:
# Fill NaNs with 0 values to keep dataset at 61001 examples for grader
df_test['hearing_date'].fillna(0, inplace=True)

In [34]:
from sklearn import preprocessing

# Create categorical variable for agency_name

le3 = preprocessing.LabelEncoder()
an3 = df_test['agency_name'].tolist()
le_model3 = le3.fit_transform(an3)

#print(type(le_model))

df_test['agency_name'] = le_model3
#df_test['agency_name'].unique()


# Create categorical variable for disposition
le4 = preprocessing.LabelEncoder()
dis4 = df_test['disposition'].tolist()
le_model4 = le4.fit_transform(dis4)

df_test['disposition'] = le_model4
#df_test['disposition'].unique()

In [35]:
df_test['ticket_issued_date'].head()

ticket_id
284932    2012-01-04 14:00:00
285362    2012-01-05 09:50:00
285361    2012-01-05 09:50:00
285338    2012-01-05 10:25:00
285346    2012-01-05 10:20:00
Name: ticket_issued_date, dtype: object

In [36]:
# convert to int to find the number of days from ticket_issued_date to the hearing_date
df_test['time_to_hearing_date'] = (pd.to_datetime(df_test['hearing_date']).dt.date - 
                     pd.to_datetime(df_test['ticket_issued_date']).dt.date).fillna(pd.Timedelta('-1 days')).dt.days

In [37]:
df_test['time_to_hearing_date'].head()

ticket_id
284932    15
285362    32
285361    32
285338    33
285346    40
Name: time_to_hearing_date, dtype: int64

In [38]:
# Set up X_test for test dataset
X_test = df_test[['time_to_hearing_date','fine_amount', 'admin_fee','state_fee', 'late_fee','discount_amount', 'judgment_amount']]

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# instantiate the logistic model
clf = LogisticRegression()

logreg_model = clf.fit(X_train, y_train)

In [40]:
# Calculate the predicted values
y_pred = logreg_model.predict(X_train)

In [41]:
# Calculate roc_auc score for training model
x = roc_auc_score(y_train,y_pred)  
x

0.94174687920170186

In [42]:
# Obtain the predicted probabilities for the X_test dataset
y_pred_proba = logreg_model.predict_proba(X_test)[:, 1]

# convert to list
list_predicted_probabilities = list(y_pred_proba)

# convert to panda series
series_predicted_probabilities = pd.Series(list_predicted_probabilities).astype('float32')

/Users/karenyang/anaconda/lib/python3.6/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [43]:
# add values of series to dataframe
df_test['compliance'] = series_predicted_probabilities.values

df_final = df_test[['compliance']]
df_final.head()

,compliance
ticket_id,
284932,0.062065
285362,0.007397
285361,0.078228
285338,0.060755
285346,0.077503


In [44]:
answer = df_final['compliance'] 
answer[500:510]  # sample of predicted probabilities for test data
#return answer

ticket_id
286183    0.061334
288330    0.106148
286909    0.087834
286908    0.007333
286102    0.061919
287151    0.059825
288422    0.082332
286426    0.028085
286430    0.028085
286424    0.187098
Name: compliance, dtype: float32

Grader output:

Your AUC of  0.759420902783 was awarded a value of 1.0 out of  1.0 total grades

The point of a ROC curve (and the area under it) is that you study the precision-recall tradeoff as the 
classification threshold is varied. By default in a binary classification task, if your classifier's score is  > 0.5, 
then class1 is predicted, otherwise class0 is predicted. 